In [ ]:
import os
import csv
import yaml
import wandb
import pickle
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.utils import to_categorical
from keras.layers import Dense
from keras.models import Sequential
from src import (
    read_nz_file, read_jg_file, update_meta_data, split_df, aggregate_files, add_moving_window, add_moving_window_2,
    preprocess_data, save_preprocessing, one_hot_encode, preprocess_sequential, save_sequential_preprocessing)
from sklearn.model_selection import train_test_split
from wandb.keras import WandbCallback

## load and show meta data of recorded activities

In [ ]:
update_meta_data()

meta = pd.read_csv('data/meta.csv')
meta

## run sequential preprocessing

In [ ]:
# for moving_window_size in [1, 2]:
#     for hz in [1, 2, 5, 10, 20]:

settings = {
    'MOVING_WINDOW_SIZE': 2,
    'HZ': 5,
    'STEP_SIZE': 5,
    'TEST_PROPORTION': 0.5,
    'AGGREGATION': "sequential",
    'FEATURES': "min, max, EucDist"
}

directory = f"sequential_index_{settings['HZ']}hz_{settings['MOVING_WINDOW_SIZE']}sec"
print(directory)

X_train, X_test, y_train, y_test, train_indexes, test_indexes = preprocess_sequential(
    moving_window_seconds = settings['MOVING_WINDOW_SIZE'],
    hz = settings['HZ'],
    step_size = settings['STEP_SIZE'],
    meta = meta,
    test_proportion = settings['TEST_PROPORTION'],
    select_train_files = 'index'
)

save_sequential_preprocessing(X_train, X_test, y_train, y_test, train_indexes, test_indexes, directory, settings)

## run classical preprocessing

In [ ]:
directory = 'dimension_independent_5hz_2sec'

settings = {
    'MOVING_WINDOW_SIZE': 2,
    'HZ': 5,
    'STEP_SIZE': 5,
    'TEST_PROPORTION': 0.2,
    'AGGREGATION': "normal",
    'FEATURES': "min, max, EucDist (mean & std)",
    'PREPROCESSING': directory
}

X_train, X_test, y_train, y_test = preprocess_data(
    moving_window_seconds = settings['MOVING_WINDOW_SIZE'],
    hz = settings['HZ'],
    step_size = settings['STEP_SIZE'],
    meta = meta,
    test_proportion = settings['TEST_PROPORTION'],
    agg_func = 'add_moving_window_2',
    select_train_files='user' # 'all', 'index', 'position_x'
)

save_preprocessing(X_train, X_test, y_train, y_test, directory, settings)